# GRO620 - Activité procédurale 3

In [ ]:
# Préambule

import numpy as np
import cv2
import math

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive

## Filtrage linéaire - suite

### Q1.1

Soit cette image :

In [ ]:
img_q11_org  = cv2.imread("images_doc/proc2-q1-object.jpeg")
img_q11_mono = np.float32(cv2.cvtColor(img_q11_org, cv2.COLOR_BGR2GRAY)) / 255.
plt.imshow(img_q11_mono, cmap="gray")

Utilisez un filtre linéaire de Sobel (voir sections 3.2.1 et 4.2.1) pour faire ressortir les contours de cette image.

In [ ]:
sobel = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]]) * (1/8.)

img_sob = cv2.filter2D(img_q11_mono, -1, sobel)

plt.imshow(img_sob)


## Filtrage non-linéaire

### Q2.1

Soit cette image :

In [ ]:
img_q21_org = cv2.imread("images_doc/proc2-q3-sand.png")
img_q21_mono = cv2.cvtColor(img_q21_org, cv2.COLOR_BGR2RGB)
plt.imshow(img_q21_mono, cmap="gray")

Utilisez un filtre **non-linéaire** pour tenter de retirer les points noirs dans l'image.

In [ ]:
img_q21_filt = img_q21_mono.copy()
img_q21_median = cv2.medianBlur(img_q21_filt,5);
plt.imshow(img_q21_median, cmap="gray")

###Méthode 2### PLUS POUR DES IMAGES AVEC CONTOUR
#img_q21_bi = cv2.bilateralFilter(img_q21_filt,10,80,80, cv2.BORDER_DEFAULT);
#plt.imshow(img_q21_bi, cmap="gray")

## Chaîne de filtrage

### Q3.1

En reprenant la figure originale de la question Q1.1, appliquez une succession de filtres pour mieux isoler le contour de l'objet au centre de l'image.

In [ ]:
img_q31 = img_q11_mono
plt.imshow(img_q31, cmap="gray")
plt.figure()

img_q31_filter = cv2.GaussianBlur(img_q31,(1,1),0)

img_q31_clean = cv2.Sobel(img_q31_filter, -1, 0, 1)

plt.imshow(img_q31_clean)

### Q3.2

Répétez l'exercice précédent, mais avec un filtre de Canny.

In [ ]:
img_q32 = img_q11_org

img_q31_filter = cv2.GaussianBlur(img_q11_org,(15,15),0)

img_q31_clean = cv2.Canny(img_q31_filter, 150, 250)
plt.imshow(img_q31_clean)

## Analyse de formes

### Q4.1

Soit cette image :

In [ ]:
img_q41_org  = cv2.imread("images_doc/proc3-q4-tracks.jpeg")
img_q41_rgb = cv2.cvtColor(img_q41_org, cv2.COLOR_BGR2RGB)
plt.imshow(img_q41_rgb)

(Source de l'image originale : [Krivec Ales, Pexels](https://www.pexels.com/photo/clouds-cloudy-country-distance-552779/))

Trouvez et tracez les lignes dominantes à l'aide de la transformée de Hough.

In [ ]:
img_q41_mono = cv2.cvtColor(img_q41_rgb, cv2.COLOR_BGR2GRAY)
img_q41_filt = cv2.Canny(img_q41_mono,150,250)
plt.imshow(img_q41_filt)
lines = cv2.HoughLines(img_q41_filt, 1, np.pi / 180, 180, None, 0, 0);
img_q41_filt = cv2.cvtColor(img_q41_filt, cv2.COLOR_GRAY2BGR)
for i in range(0, len(lines)):
    rho = lines[i][0][0]
    theta = lines[i][0][1]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv2.line(img_q41_filt, pt1, pt2, (0,0,255), 3, cv2.LINE_AA)
plt.figure()    
plt.imshow(img_q41_filt, cmap="gray")

### Q4.2

En reprenant l'image des questions Q1.1 et Q3.1, trouvez, tracez et afficher les coordonnées (en pixels) de la boîte (*bounding box*) contenant les contours de l'objet à l'aide de cv2.findContours()

In [ ]:
img_q41_out = img_q11_org.copy() # Pour le résultat
img_q41_filter = cv2.GaussianBlur(img_q41_out,(15,15),0)
img_q41_f = cv2.Canny(img_q41_filter,150,250)
plt.imshow(img_q41_f)
plt.figure()
contours, hierarchy = cv2.findContours(img_q41_f,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours(img_q41_out, contours, 0, (255, 230, 0), 2)
x,y,w,h = cv2.boundingRect(contours[0])
print(w,h)
cv2.rectangle(img_q41_out, (x,y), (x+w,y+h), (0,255,0),2)
cv2.circle(img_q41_out, (x+int(w/2), y+int(h/2)), 3,(0,0,255),-1) 
plt.imshow(img_q41_out)